In [19]:
train = '''939
7,13,x,x,59,x,31,19'''

test = '''1001796
37,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,41,x,x,x,x,x,x,x,x,x,457,x,x,x,x,x,x,x,x,x,x,x,x,13,17,x,x,x,x,x,x,x,x,23,x,x,x,x,x,29,x,431,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,19'''

class ShuttleBuses():
    
    def __init__(self, notes):

        earliest_ts, bus_ids = notes.split('\n')

        self.earliest_ts = int(earliest_ts)

        self.bus_ids = pd.Series(data=bus_ids.split(','), name='bus_id').map(lambda x: np.nan if x == 'x' else int(x))
        
    def find_earliest_bus(self):
        
        next_departures = {
            int(bus_id): 0
            for bus_id
            in self.bus_ids.dropna()}

        ts = self.earliest_ts

        while True:

            for bus_id in self.bus_ids.dropna():
                while next_departures[bus_id] < ts:
                    next_departures[bus_id] += bus_id

                if ts == next_departures[bus_id]:
                    return({
                        'ts':     ts,
                        'bus_id': bus_id})

            ts += 1
            
    def find_contest_solution(self):
        
        def _get_wave(index):

            bus_id = self.bus_ids[index]

            if np.isnan(bus_id):
                return {
                    'start':  0,
                    'length': 1}

            return {
                'start':  bus_id - index,
                'length': bus_id}
        
        def _combine_wave(wave_1, wave_2):

            start_1  = wave_1['start']
            start_2  = wave_2['start']
            
            length_1 = wave_1['length']
            length_2 = wave_2['length']

            max_length = max(length_1, length_2)
            min_length = min(length_1, length_2)

            start       = {length_1: start_1, length_2: start_2}[max_length]
            other_start = {length_1: start_1, length_2: start_2}[min_length]

            while (start - other_start) % min_length != 0:
                start += max_length

            gcd    = math.gcd(length_1, length_2)
            length = int(length_1 * length_2 / gcd)

            return {
                'start':  start,
                'length': length}

        df = pd.DataFrame({
            index: _get_wave(index)
            for index
            in self.bus_ids.index}).T.astype(int)

        wave = {
            'start':  0,
            'length': 1}

        for row in df.itertuples():
            wave_2 = {
                'start':  row.start,
                'length': row.length}
            wave = _combine_wave(wave_1=wave, wave_2=wave_2)

        return wave
    
self = ShuttleBuses(notes=train)
data = self.find_earliest_bus()
assert (data['ts'] - self.earliest_ts) * data['bus_id'] == 295
assert self.find_contest_solution()['start'] == 1068781

self = ShuttleBuses(notes=test)
data = self.find_earliest_bus()
(data['ts'] - self.earliest_ts) * data['bus_id']
self.find_contest_solution()['start']

261.0

807435693182510